In [1]:
import os
import pandas as pd
from datasets import load_dataset
from transformers import GPT2TokenizerFast
import torch
import hopsworks
from sklearn.model_selection import train_test_split

c:\Users\farha\anaconda3\envs\news-sentiment\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
tokenizer = GPT2TokenizerFast.from_pretrained('Xenova/text-embedding-ada-002')

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT4Tokenizer'. 
The class this function is called from is 'GPT2TokenizerFast'.


## Functions

In [3]:
def load_data(file_path):
    data = pd.read_csv(file_path)
    return data

def get_embedding(dataset, embedding_object):
    embeddings = []
    for data in dataset["text"]:
        embedded_text = embedding_object.encode(data)
        embeddings.append(embedded_text)

    dataset_embedded = dataset.copy()
    dataset_embedded["embeddings"] = embeddings
    dataset_embedded = dataset_embedded.drop(columns=["text"])
    return dataset_embedded

In [4]:
financial_phrase_bank_df = load_data(os.path.join("base-data", "FinancialPhraseBank", "all-data-75-above.csv"))
zeroshot_train_df = load_data(os.path.join("base-data", "twitter-financial-news-sentiment", "sent_train.csv"))
zeroshot_test_df = load_data(os.path.join("base-data", "twitter-financial-news-sentiment", "sent_test.csv"))

In [5]:

# Assuming df1, df2, df3 are your dataframes
df = pd.concat([financial_phrase_bank_df, zeroshot_train_df, zeroshot_test_df])

# Get the count of each label
label_counts = df['label'].value_counts()

# Split the data
X_train, X_test, y_train, y_test = train_test_split(df['text'], df['label'], test_size=0.2, stratify=df['label'])

# Now, X_train and y_train contain the training data and their corresponding labels
# X_test and y_test contain the test data and their corresponding labels

## Sanity check of label distribution

In [6]:
y_train_df = pd.DataFrame(y_train)
train_value_counts= y_train_df.value_counts()
# Get the percentage of each label in the training data
print(train_value_counts / train_value_counts.sum())

y_test_df = pd.DataFrame(y_test)
test_value_counts= y_test_df.value_counts()
print(test_value_counts / test_value_counts.sum())


label
2        0.642886
1        0.213537
0        0.143577
Name: count, dtype: float64
label
2        0.642834
1        0.213520
0        0.143646
Name: count, dtype: float64


In [7]:
train_dataset_df = pd.concat([X_train, y_train], axis=1)
test_dataset_df = pd.concat([X_test, y_test], axis=1)

train_dataset_df_embedded = get_embedding(train_dataset_df, tokenizer)
test_dataset_df_embedded = get_embedding(test_dataset_df, tokenizer)

train_dataset_df_embedded

,label,embeddings
2984,2,"[82478, 427, 309, 14966, 364, 82, 33956, 12466..."
2395,2,"[16648, 279, 3568, 1174, 14521, 8871, 690, 234..."
187,0,"[33, 41715, 12, 5159, 388, 2795, 9269, 315, 10..."
626,2,"[3, 34, 1112, 47, 482, 356, 13136, 11, 4953, 1..."
6559,2,"[39320, 819, 41420, 24190, 2019, 24854, 690, 2..."
...,...,...
4875,2,"[32723, 40, 11977, 682, 15167, 449, 469, 7939,..."
1646,2,"[2822, 21913, 3649, 1051, 36489, 662]"
6182,0,"[59562, 94963, 13020, 92887, 9511, 3157, 4787,..."
105,0,"[74872, 35374, 1523, 24228, 311, 4662, 505, 37..."


In [8]:
hopsworks_project = hopsworks.login() 
fs = hopsworks_project.get_feature_store()

Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/197784
Connected. Call `.close()` to terminate connection gracefully.


In [9]:
fg_train = fs.get_or_create_feature_group(name="news_sentiment_traindata", version=1, description="Training data and labels for financial news sentiment prediction model", primary_key=["label", "embeddings"], online_enabled=True)
fg_train.insert(train_dataset_df_embedded)

Feature Group created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/197784/fs/197703/fg/332115


Uploading Dataframe: 100.00% |██████████| Rows 12307/12307 | Elapsed Time: 00:08 | Remaining Time: 00:00


Launching job: news_sentiment_traindata_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/197784/jobs/named/news_sentiment_traindata_1_offline_fg_materialization/executions


(<hsfs.core.job.Job at 0x1f6aea6fc70>, None)

In [10]:
fg_test = fs.get_or_create_feature_group(name="news_sentiment_testdata", version=1, description="Test data and labels for financial news sentiment prediction model", primary_key=["label", "embeddings"], online_enabled=True)
fg_test.insert(test_dataset_df_embedded)

Feature Group created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/197784/fs/197703/fg/330098


Uploading Dataframe: 100.00% |██████████| Rows 3077/3077 | Elapsed Time: 00:06 | Remaining Time: 00:00


Launching job: news_sentiment_testdata_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/197784/jobs/named/news_sentiment_testdata_1_offline_fg_materialization/executions


(<hsfs.core.job.Job at 0x1f6afd7ae80>, None)